<a href="https://colab.research.google.com/github/aditisingh2912/Data-Mining/blob/main/IISc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Assignment

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1gro73TIO2rzqpkcjGSJSOjt-4NFYc-_5/Assignment


In [3]:
!pip install -r requirements.txt

  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached pandas-2.0.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached torch-2.0.1-cp311-cp311-manylinux1_x86_64.whl.metadata (24 kB)
  Using cached scikit_learn-1.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement faiss==1.7.4 (from versions: none)
ERROR: No matching distribution found for faiss==1.7.4


In [4]:
import sys
sys.path.append('./src')

In [5]:
import os
import sys
import torch
import numpy as np
import pandas as pd
from PIL import Image

sys.path.append('./src')
from models import RED_DOT


In [6]:
!pip install salesforce-lavis==1.0.2

In [8]:
# ---- SETTINGS ----
from lavis.models import load_model_and_preprocess
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
CHECKPOINT_PATH = "/content/drive/MyDrive/Assignment/checkpoints_pt/best_model.pt"  # Adjust file name
TEST_DATA_DIR = "/content/drive/MyDrive/Assignment/Test Data"
NUM_SAMPLES = 10

# ---- MODEL SETUP ----
model = RED_DOT(
    device=DEVICE,
    emb_dim=768,          # Changed to match checkpoint
    tf_layers=4,
    tf_head=8,
    tf_dim=128,
    use_evidence=0,
    use_neg_evidence=0,
    model_version="baseline" # Assuming the model version in the checkpoint is 'baseline' or compatible
)
checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
# Assuming the state dictionary is directly in the checkpoint, otherwise adjust the key (e.g., checkpoint['model_state_dict'])
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

# ---- CLIP SETUP ----
clip_model, vis_processors, txt_processors = load_model_and_preprocess(
    name="clip_feature_extractor",
    model_type="ViT-L-14",
    is_eval=True,
    device=DEVICE
)

/usr/local/lib/python3.11/dist-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/usr/local/lib/python3.11/dist-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [9]:
print(next(model.parameters()).mean())

tensor(0.0624, grad_fn=<MeanBackward0>)


In [10]:
for name, param in model.named_parameters():
    print(name, param.mean(), param.std())

cls_token tensor(0.0624, grad_fn=<MeanBackward0>) tensor(1.0293, grad_fn=<StdBackward0>)
transformer.layers.0.self_attn.in_proj_weight tensor(2.9401e-06, grad_fn=<MeanBackward0>) tensor(0.0326, grad_fn=<StdBackward0>)
transformer.layers.0.self_attn.in_proj_bias tensor(-0.0002, grad_fn=<MeanBackward0>) tensor(0.0214, grad_fn=<StdBackward0>)
transformer.layers.0.self_attn.out_proj.weight tensor(3.4457e-05, grad_fn=<MeanBackward0>) tensor(0.0264, grad_fn=<StdBackward0>)
transformer.layers.0.self_attn.out_proj.bias tensor(0.0002, grad_fn=<MeanBackward0>) tensor(0.0399, grad_fn=<StdBackward0>)
transformer.layers.0.linear1.weight tensor(-2.2577e-05, grad_fn=<MeanBackward0>) tensor(0.0271, grad_fn=<StdBackward0>)
transformer.layers.0.linear1.bias tensor(-0.0129, grad_fn=<MeanBackward0>) tensor(0.0267, grad_fn=<StdBackward0>)
transformer.layers.0.linear2.weight tensor(-2.5483e-05, grad_fn=<MeanBackward0>) tensor(0.0538, grad_fn=<StdBackward0>)
transformer.layers.0.linear2.bias tensor(0.0020, g

/tmp/ipython-input-10-4226696417.py:2: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  print(name, param.mean(), param.std())


In [11]:
print(model.binary_classifier.output_layer.bias)

Parameter containing:
tensor([0.0082], requires_grad=True)


In [12]:
results = []
for i in range(1, NUM_SAMPLES+1):
    sample_dir = os.path.join(TEST_DATA_DIR, f"sample{i}")
    image_path = os.path.join(sample_dir, "Image.jpg")
    caption_path = os.path.join(sample_dir, "caption.txt")
    gt_path = os.path.join(sample_dir, "GT.txt")

    image = Image.open(image_path).convert("RGB")
    with open(caption_path, "r") as f:
        caption = f.read().strip()
    with open(gt_path, "r") as f:
        gt_str = f.read().strip().lower()
        if gt_str == "fake":
            ground_truth = 1
        elif gt_str == "true":
            ground_truth = 0
        else:
            raise ValueError(f"Unknown label: {gt_str} in {gt_path}")


In [ ]:
txt_obj = txt_processors["eval"](caption)
print("txt_obj type:", type(txt_obj))
print("txt_obj:", txt_obj)

In [13]:
!pip install clip
!pip install git+https://github.com/openai/CLIP.git
import clip
import torch
tokenized = clip.tokenize([caption]).to(DEVICE)  # Always batch!
txt_tensor = tokenized

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-u6ywrzux
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-u6ywrzux
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [14]:
CHECKPOINT_PATH = "/content/drive/MyDrive/Assignment/checkpoints_pt/best_model.pt"
checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [34]:
results = []
for i in range(1, NUM_SAMPLES + 1):
    sample_dir = os.path.join(TEST_DATA_DIR, f"sample{i}")
    image_path = os.path.join(sample_dir, "Image.jpg")
    caption_path = os.path.join(sample_dir, "caption.txt")
    gt_path = os.path.join(sample_dir, "GT.txt")

    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")
    img_tensor = vis_processors["eval"](image).unsqueeze(0).to(DEVICE)

    # Load and tokenize caption
    with open(caption_path, "r") as f:
        caption = f.read().strip()
    txt_tensor = clip.tokenize([caption]).to(DEVICE)
    if isinstance(txt_tensor, tuple):
        txt_tensor = txt_tensor[0]
    if txt_tensor.ndim == 1:
        txt_tensor = txt_tensor.unsqueeze(0)
    txt_tensor = txt_tensor.to(DEVICE)

    # Load ground truth
    with open(gt_path, "r") as f:
        gt_str = f.read().strip().lower()
        if gt_str == "fake":
            ground_truth = 1
        elif gt_str == "true":
            ground_truth = 0
        else:
            raise ValueError(f"Unknown label: {gt_str} in {gt_path}")



    with torch.no_grad():
        model.load_state_dict(checkpoint["model_state_dict"])
        model.to(DEVICE)
        model.eval()
        img_features = clip_model.encode_image(img_tensor)
        txt_features = clip_model.encode_text(txt_tensor)
        features = torch.stack([img_features, txt_features], dim=1)
        features = features.to(DEVICE)
        print(f"Sample {i} features shape:", features.shape)  # Debug

        output = model(features, inference=True)
        if isinstance(output, tuple):
            logits = output[0]
        else:
            logits = output

        probs = torch.sigmoid(logits).cpu().numpy().flatten()
        confidence = float(np.max([probs, 1-probs]))
        entropy = float(-(probs * np.log(probs + 1e-8) + (1-probs) * np.log(1-probs + 1e-8)))
        pred_label = int(probs < 0.5)

    results.append({
        "sample": i,
        "probability": probs[0],
        "logit": logits.cpu().numpy().flatten()[0],
        "ground_truth": ground_truth
    # optionally add: "predicted_label": int(probs > 0.5)
})
df = pd.DataFrame(results)

Sample 1 features shape: torch.Size([1, 2, 768])
Sample 2 features shape: torch.Size([1, 2, 768])
Sample 3 features shape: torch.Size([1, 2, 768])


/tmp/ipython-input-34-2721450554.py:52: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  entropy = float(-(probs * np.log(probs + 1e-8) + (1-probs) * np.log(1-probs + 1e-8)))
/tmp/ipython-input-34-2721450554.py:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_label = int(probs < 0.5)


Sample 4 features shape: torch.Size([1, 2, 768])
Sample 5 features shape: torch.Size([1, 2, 768])
Sample 6 features shape: torch.Size([1, 2, 768])
Sample 7 features shape: torch.Size([1, 2, 768])
Sample 8 features shape: torch.Size([1, 2, 768])
Sample 9 features shape: torch.Size([1, 2, 768])
Sample 10 features shape: torch.Size([1, 2, 768])


In [35]:
for i in range(1, NUM_SAMPLES + 1):
  print(f"Sample {i}: img_features mean={img_features.mean().item()}, std={img_features.std().item()}")
  print(f"Sample {i}: txt_features mean={txt_features.mean().item()}, std={txt_features.std().item()}")

Sample 1: img_features mean=-0.010069314390420914, std=0.6967156529426575
Sample 1: txt_features mean=-0.01145151536911726, std=0.4450313448905945
Sample 2: img_features mean=-0.010069314390420914, std=0.6967156529426575
Sample 2: txt_features mean=-0.01145151536911726, std=0.4450313448905945
Sample 3: img_features mean=-0.010069314390420914, std=0.6967156529426575
Sample 3: txt_features mean=-0.01145151536911726, std=0.4450313448905945
Sample 4: img_features mean=-0.010069314390420914, std=0.6967156529426575
Sample 4: txt_features mean=-0.01145151536911726, std=0.4450313448905945
Sample 5: img_features mean=-0.010069314390420914, std=0.6967156529426575
Sample 5: txt_features mean=-0.01145151536911726, std=0.4450313448905945
Sample 6: img_features mean=-0.010069314390420914, std=0.6967156529426575
Sample 6: txt_features mean=-0.01145151536911726, std=0.4450313448905945
Sample 7: img_features mean=-0.010069314390420914, std=0.6967156529426575
Sample 7: txt_features mean=-0.0114515153691

In [36]:
for i in range(1, NUM_SAMPLES + 1):
    sample_dir = os.path.join(TEST_DATA_DIR, f"sample{i}")
    image_path = os.path.join(sample_dir, "Image.jpg")
    caption_path = os.path.join(sample_dir, "caption.txt")
    gt_path = os.path.join(sample_dir, "GT.txt")
    results.append({
    "sample": i,
    "probability": probs[0],
    "logit": logits.cpu().numpy().flatten()[0],
    "ground_truth": ground_truth
    # optionally add: "predicted_label": int(probs > 0.5)
})
    print(f"Sample {i}: logits={logits}")

Sample 1: logits=tensor([[-7.8397]], device='cuda:0')
Sample 2: logits=tensor([[-7.8397]], device='cuda:0')
Sample 3: logits=tensor([[-7.8397]], device='cuda:0')
Sample 4: logits=tensor([[-7.8397]], device='cuda:0')
Sample 5: logits=tensor([[-7.8397]], device='cuda:0')
Sample 6: logits=tensor([[-7.8397]], device='cuda:0')
Sample 7: logits=tensor([[-7.8397]], device='cuda:0')
Sample 8: logits=tensor([[-7.8397]], device='cuda:0')
Sample 9: logits=tensor([[-7.8397]], device='cuda:0')
Sample 10: logits=tensor([[-7.8397]], device='cuda:0')


In [37]:
df

,sample,probability,logit,ground_truth
0,1,0.272330,-0.982835,1
1,2,0.000153,-8.783122,0
2,3,0.062082,-2.715200,1
3,4,0.000045,-10.011206,1
4,5,0.000157,-8.757331,0
5,6,0.044764,-3.060564,1
6,7,0.001595,-6.439342,1
7,8,0.000082,-9.411135,0
8,9,0.002630,-5.937966,0
9,10,0.000394,-7.839696,1
